In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import AdaBoostRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import StackingRegressor
sns.set(rc={'figure.figsize':(15,10)})
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error

In [ ]:
df = pd.read_csv('/kaggle/input/top50spotify2019/top50.csv',encoding='ISO-8859-1')
df.set_index('Unnamed: 0',inplace=True)
df.head()

In [ ]:
df.rename(columns={'Track.Name' : 'Track', 
                     'Artist.Name' : 'Artist', 
                     'Genre': 'Genre', 
                     'Beats.Per.Minute' : 'BPM',
                     'Energy' : 'Energy', 
                     'Danceability' : 'Dance', 
                     'Loudness..dB..' : 'Loudness(dB)', 
                     'Liveness' : 'Liveness', 
                     'Valence.' : 'Valence',
                     'Length.' : 'Length', 
                     'Acousticness..' : 'Acoustic', 
                     'Speechiness.' : 'Speech', 
                     'Popularity' : 'Popularity'}, inplace = True)
df.head()

In [ ]:
df.info()

In [ ]:
df.isna().sum()

In [ ]:
df.describe()

In [ ]:
df['Genre'].value_counts()

# EDA

lets see the correlation between the features.

In [ ]:
sns.heatmap(df.corr(),annot=True,cmap='BuPu')

**Plotting differnt features against Popularity**

In [ ]:
sns.barplot(x='Genre',y='Popularity',data=df,palette='rocket_r')
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.barplot(x='Artist',y='Popularity',data=df,palette='magma_r')
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.scatterplot(x='BPM',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Loudness(dB)',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Liveness',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Dance',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Length',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Speech',y='Popularity',hue='Energy',data=df,palette='rocket_r')

In [ ]:
sns.scatterplot(x='Acoustic',y='Popularity',hue='Energy',data=df,palette='rocket_r')

let us see the average of different features grouped together against Genres of songs from the dataset.

In [ ]:
g_p = df.groupby('Genre')['Popularity','BPM','Liveness','Length'].mean()
g_p

In [ ]:
g_p.plot(kind='bar')

Repeating the same process for different Artists from the dataset.

In [ ]:
a_p = df.groupby('Artist')['Popularity','BPM','Dance','Length'].mean()
a_p

In [ ]:
a_p[0:10].plot(kind='bar')

In [ ]:
a_p[10:20].plot(kind='bar')

In [ ]:
a_p[20:30].plot(kind='bar')

In [ ]:
a_p[30:].plot(kind='bar')

# MODELLING

Trying to predict the Popularity of a song by training a model on different features.

In [ ]:
enc = LabelEncoder()
df['Artist_enc'] = enc.fit_transform(df['Artist'])
df['Genre_enc'] = enc.fit_transform(df['Genre'])
df

In [ ]:
target = df['Popularity']
train = df.drop(['Popularity','Artist','Genre','Track'],axis=1)
print(train.shape,target.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.1, random_state=42,shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X_train)
scaler.transform(X_test)

In [ ]:
#models
rf = RandomForestRegressor(bootstrap=True,criterion='mse',random_state=42,max_depth=35,
                           max_features=11,n_estimators=2500,n_jobs=-1)
xgb = XGBRegressor(booster='gbtree', colsample_bylevel=1,colsample_bynode=1, colsample_bytree=0.6, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=6, min_child_weight=4, n_estimators=4500,
             n_jobs=4, nthread=None, objective='reg:squarederror',
             reg_alpha=0.6, reg_lambda=0.6, scale_pos_weight=1, 
             silent=None, subsample=0.8, verbosity=1)
gbr = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.01, max_depth=4,
                                max_features='sqrt', min_samples_leaf=15, min_samples_split=10,
                                loss='huber', random_state =42) 
dtr = DecisionTreeRegressor(criterion='mse',random_state=42,max_depth=35,
                           max_features='sqrt', min_samples_leaf=15, min_samples_split=10)
abr = AdaBoostRegressor(dtr,n_estimators=4500,random_state=42,learning_rate=0.01)
cat =  CatBoostRegressor(depth=6,learning_rate=0.1,n_estimators=2500,eval_metric = 'RMSE')
lgb = LGBMRegressor(max_depth=25,num_leaves=120,learning_rate=0.01,n_jobs=-1,boosting_type='gbdt',
                   objective='regression',mertic='rmse', verbosity=1,bagging_fraction=0.7,
                   feature_fraction=0.5,bagging_frequency=6,bagging_seed=42,seed=42,
                   colsample_bylevel=1,colsample_bynode=1, colsample_bytree=1)

In [ ]:
xgb.fit(X_train,y_train)
xgb_pred = xgb.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,xgb_pred)),mean_absolute_error(y_test,xgb_pred))

In [ ]:
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,rf_pred)),mean_absolute_error(y_test,rf_pred))

In [ ]:
dtr.fit(X_train,y_train)
dtr_pred = dtr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,dtr_pred)),mean_absolute_error(y_test,dtr_pred))

In [ ]:
abr.fit(X_train,y_train)
abr_pred = abr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,abr_pred)),mean_absolute_error(y_test,abr_pred))

In [ ]:
gbr.fit(X_train,y_train)
gbr_pred = gbr.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,gbr_pred)),mean_absolute_error(y_test,gbr_pred))

In [ ]:
cat.fit(X_train,y_train)
cat_pred = cat.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,cat_pred)),mean_absolute_error(y_test,cat_pred))

In [ ]:
lgb.fit(X_train,y_train)
lgb_pred = lgb.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,lgb_pred)),mean_absolute_error(y_test,lgb_pred))

In [ ]:
VR = VotingRegressor([('xgb',xgb),('lgb',lgb),('gbr',gbr),('cat',cat),('dtr',dtr)],n_jobs=-1)
VR.fit(X_train,y_train)
VR_pred = VR.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,VR_pred)),mean_absolute_error(y_test,VR_pred))

In [ ]:
level0 = [('xgb',xgb),('lgb',lgb),('gbr',gbr),('abr',abr),('cat',cat),('vr',VR)]
level1 = LinearRegression()
stack = StackingRegressor(estimators=level0, cv=5,n_jobs=-1)
stack.fit(X_train,y_train)
stack_pred = stack.predict(X_test)
print(np.sqrt(mean_squared_error(y_test,stack_pred)),mean_absolute_error(y_test,stack_pred))

In [ ]:
output = pd.DataFrame({'Actual':y_test,'Predicted_xgb':xgb_pred,'Predicted_lgb':lgb_pred,
                      'Predicted_abr':abr_pred,'Predicted_gbr':gbr_pred,'Predicted_dtr':dtr_pred,
                      'Predicted_rf':rf_pred,'Predicted_cat':cat_pred,
                      'Predicted_VR':VR_pred,'Predicted_stack':stack_pred})
output

In [ ]:
output.to_csv('output.csv',index=False)